# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
import openai

openai.api_key = OPENAI_API_KEY

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0): 
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, good sir! Verily, a classic jest amongst the poultry kind.


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have the capability to remember specific information about individual users. How can I assist you today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [7]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import openai
import os
openai.api_key = os.environ["OPENAI_API_KEY"]


# Create widgets
input_box = widgets.Text(placeholder='Enter your message...')
send_button = widgets.Button(description='Send')
output_area = widgets.Output()

# Initialize conversation context
context = [{'role': 'system', 'content': """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collect the order, \
and then ask if it's for pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally, you collect the payment. \
Make sure to clarify all options, extras, and sizes to uniquely \
identify the item from the menu. \
The menu includes \
pepperoni pizza 12.95, 10.00, 7.00 \
cheese pizza 10.95, 9.25, 6.50 \
eggplant pizza 11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00 \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""}]

def on_button_click(b):
    user_message = input_box.value
    if user_message.strip():
        context.append({'role': 'user', 'content': user_message})
        
        # Generate assistant's response
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',  # Use 'gpt-4' if you have access
            messages=context
        )
        
        assistant_message = response['choices'][0]['message']['content']
        context.append({'role': 'assistant', 'content': assistant_message})
        
        # Display the conversation
        with output_area:
            clear_output(wait=True)
            for message in context[1:]:  # Skip the system message
                if message['role'] == 'user':
                    print(f"User: {message['content']}")
                elif message['role'] == 'assistant':
                    print(f"Assistant: {message['content']}")
    
    input_box.value = ''  # Clear the input box

# Link the button click event to the function
send_button.on_click(on_button_click)

# Display the widgets
display(input_box, send_button, output_area)

Text(value='', placeholder='Enter your message...')

Button(description='Send', style=ButtonStyle())

Output()

In [44]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "pizza": {
        "type": "pepperoni pizza",
        "size": "large"
    },
    "toppings": [
        "extra cheese",
        "mushrooms"
    ],
    "drinks": [
        {
            "type": "coke",
            "size": "medium"
        },
        {
            "type": "sprite",
            "size": "small"
        }
    ],
    "sides": [
        {
            "type": "fries",
            "size": "regular"
        }
    ],
    "total price": 27.45
}
```


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [9]:
# VERSION 1 FURRY BALL BOT
openai.api_key = os.environ["OPENAI_API_KEY"]


# Create widgets
input_box = widgets.Text(placeholder='Enter your message...')
send_button = widgets.Button(description='Send')
output_area = widgets.Output()

# Initialize conversation context
context = [{'role': 'system', 'content': """
You are the furry ball bot, you are in charge of making appointments for pet owner that want to cut their pets hair \
You first greet the customer saying happy furry days, then you should ask when they want the bring their furry friend, \
and then ask whether if it a cat or a dog \
Then you should ask what kind of haircut they want for their pet \
time if the customer wants anything else.  \
Finally, you collect the payment. \
Make sure to clarify all options. \
identify the item from the menu. \
The menu includes \
only haircut, it will take 1 hour to complete \
haircut and shampooing, the service takes 1.5 hours\
complete service; haircut shampooing and perfume, this service will take 2 hours \
The price will depend on the size of the pet, how dirty the hair is and it ranges from 10$ to 30$ \
"""}]

def on_button_click(b):
    user_message = input_box.value
    if user_message.strip():
        context.append({'role': 'user', 'content': user_message})
        
        # Generate assistant's response
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',  # Use 'gpt-4' if you have access
            messages=context
        )
        
        assistant_message = response['choices'][0]['message']['content']
        context.append({'role': 'assistant', 'content': assistant_message})
        
        # Display the conversation
        with output_area:
            clear_output(wait=True)
            for message in context[1:]:  # Skip the system message
                if message['role'] == 'user':
                    print(f"User: {message['content']}")
                elif message['role'] == 'assistant':
                    print(f"Assistant: {message['content']}")
    
    input_box.value = ''  # Clear the input box

# Link the button click event to the function
send_button.on_click(on_button_click)

# Display the widgets
display(input_box, send_button, output_area)

Text(value='', placeholder='Enter your message...')

Button(description='Send', style=ButtonStyle())

Output()

In [10]:
#VERSION 2 SUSHI BOT
import ipywidgets as widgets
from IPython.display import display, clear_output
import openai
import os
openai.api_key = os.environ["OPENAI_API_KEY"]


# Create widgets
input_box = widgets.Text(placeholder='Enter your message...')
send_button = widgets.Button(description='Send')
output_area = widgets.Output()

# Initialize conversation context
context = [{'role': 'system', 'content': """
You are sushi bot, an automated service to collect orders for a japanese restaurant. \
You first greet the customer, then collect the order, \
and then ask if it's for pickup, delivery or eat at teh restaurant. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally, you collect the payment. \
identify the item from the menu. \
The menu includes \
Tuna nigiri 2 units for 4.95, 4 units for 5.95, 6 units for 6.95 \
Salmon nigiri 2 units for 3.95, 4 units for 4.95, 6 units for 5.95\
fried shrimp temaki 9.95 \
dragon roll 8.95 \
california roll 8.95 \
sushi burrito 12.95 \
Drinks: \
coke 3.00, \
sprite 3.00, \
bottled water 1.50 \
"""}]

def on_button_click(b):
    user_message = input_box.value
    if user_message.strip():
        context.append({'role': 'user', 'content': user_message})
        
        # Generate assistant's response
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',  # Use 'gpt-4' if you have access
            messages=context
        )
        
        assistant_message = response['choices'][0]['message']['content']
        context.append({'role': 'assistant', 'content': assistant_message})
        
        # Display the conversation
        with output_area:
            clear_output(wait=True)
            for message in context[1:]:  # Skip the system message
                if message['role'] == 'user':
                    print(f"User: {message['content']}")
                elif message['role'] == 'assistant':
                    print(f"Assistant: {message['content']}")
    
    input_box.value = ''  # Clear the input box

# Link the button click event to the function
send_button.on_click(on_button_click)

# Display the widgets
display(input_box, send_button, output_area)

Text(value='', placeholder='Enter your message...')

Button(description='Send', style=ButtonStyle())

Output()

In [11]:
#VERSION 3 ADVENURE BOT
import ipywidgets as widgets
from IPython.display import display, clear_output
import openai
import os
openai.api_key = os.environ["OPENAI_API_KEY"]


# Create widgets
input_box = widgets.Text(placeholder='Enter your message...')
send_button = widgets.Button(description='Send')
output_area = widgets.Output()

# Initialize conversation context
context = [{'role': 'system', 'content': """
You are an adventure bot, you offer adventure holidays with a twist \
You first greet the customer, then ask what kind of holiday they want water or mountain \
Then you will ask how many people are they planning to go on holiday \
then ask when on what date they plan to go on holidays \
If customer chooses mountain there are several options. \
trekking holiday starting from 75 per person \
snow and skiing holiday starting from 150 per person \
relaxing SPA in the mountains starting from 300 per person \
If they choose water there are also several options. \
Beach and mojitos starting from 75 per person per day \
Beach lobster resort starting from 100 per person per day \
UV chamber and a shower starting from 5 per person \
Then deppending on how many people they ask for, they will calculate the total price with a discount if they are more than five.\
"""}]

def on_button_click(b):
    user_message = input_box.value
    if user_message.strip():
        context.append({'role': 'user', 'content': user_message})
        
        # Generate assistant's response
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',  # Use 'gpt-4' if you have access
            messages=context
        )
        
        assistant_message = response['choices'][0]['message']['content']
        context.append({'role': 'assistant', 'content': assistant_message})
        
        # Display the conversation
        with output_area:
            clear_output(wait=True)
            for message in context[1:]:  # Skip the system message
                if message['role'] == 'user':
                    print(f"User: {message['content']}")
                elif message['role'] == 'assistant':
                    print(f"Assistant: {message['content']}")
    
    input_box.value = ''  # Clear the input box

# Link the button click event to the function
send_button.on_click(on_button_click)

# Display the widgets
display(input_box, send_button, output_area)

Text(value='', placeholder='Enter your message...')

Button(description='Send', style=ButtonStyle())

Output()